In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import svd
from scipy.optimize import least_squares

In [ ]:
pic_1_points=np.array([[553, 1234],[2647, 1292],[2834, 2601], [245, 2500],[943, 1493], [2331, 2270],[1281, 1155],[1166, 2356]])
pic_2_points=np.array([[152, 1723],[2108, 874],[2914, 1867],[569, 3032],[713, 1759],[2360, 1795], [871, 1292],[1425, 2399]])


def display_images_with_correspondences(img1, img2, points1, points2):
    """
    Displays two images side-by-side with lines connecting corresponding points.

    Args:
        img1: The first image.
        img2: The second image.
        points1: A list of points in the first image.
        points2: A list of corresponding points in the second image.
    """

    # Combine images horizontally
    combined_img = np.hstack((img1, img2))

    # Calculate offset for the second image
    offset = img1.shape[1]

    # Draw lines between corresponding points
    for p1, p2 in zip(points1, points2):
        cv2.line(combined_img, (p1[0], p1[1]), (p2[0] + offset, p2[1]), (0, 255, 0), 10)

    # Display the combined image
    plt.imshow(cv2.cvtColor(combined_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    # plt.savefig(f'Correspondence')
    plt.show()

# Example usage:
# Load images and corresponding points
img1 = cv2.imread('Pic_1.jpg')
img2 = cv2.imread('Pic_2.jpg')


# Display the images with lines
display_images_with_correspondences(img1, img2, pic_1_points, pic_2_points)

In [ ]:
def normalize_points(points):
    """
    Normalize a set of 2D points to improve numerical stability.
    """
    mean = np.mean(points, axis=0)
    std = np.std(points)
    
    T = np.array([
        [1/std, 0, -mean[0]/std],
        [0, 1/std, -mean[1]/std],
        [0, 0, 1]
    ])
    
    normalized_points = np.dot(T, np.vstack((points.T, np.ones(points.shape[0]))))
    return normalized_points[:2].T, T

def eight_point_algorithm(x, x_prime):
    """
    Computes the Fundamental Matrix F using the normalized 8-point algorithm.
    """
    # Normalize the points
    x, T = normalize_points(x)
    x_prime, T_prime = normalize_points(x_prime)
    
    # Construct matrix A for the linear equation
    A = np.zeros((len(x), 9))
    for i in range(len(x)):
        A[i] = [
            x_prime[i, 0] * x[i, 0], x_prime[i, 0] * x[i, 1], x_prime[i, 0],
            x_prime[i, 1] * x[i, 0], x_prime[i, 1] * x[i, 1], x_prime[i, 1],
            x[i, 0], x[i, 1], 1
        ]
    
    # Solve for F using SVD
    _, _, vh = svd(A)
    F = vh[-1].reshape(3, 3)
    
    # Enforce rank-2 constraint by setting the smallest singular value to 0
    u, d, vh = svd(F)
    d[-1] = 0
    F = u @ np.diag(d) @ vh
    
    # Denormalize F
    F = T_prime.T @ F @ T
    return F


F = eight_point_algorithm(pic_1_points, pic_2_points)
print("Fundamental Matrix F:\n", F)


In [ ]:

def compute_epipoles(F):

    # Compute the SVD of F
    _, _, vh = np.linalg.svd(F)
    # Right epipole (e') is the last row of V (transpose of vh)
    e_prime = vh[-1, :]
    e_prime = e_prime / e_prime[2]  # Normalize to ensure homogeneous coordinate
    
    # Left epipole (e) is the last column of U
    u, _, _ = np.linalg.svd(F.T)
    e = u[:, -1]
    e = e / e[2]  # Normalize to ensure homogeneous coordinate
    
    return e, e_prime

def compute_projection_matrices(e_prime, F):

    # Initialize P for the left image in canonical form
    P = np.array([[1, 0, 0, 0], 
                  [0, 1, 0, 0], 
                  [0, 0, 1, 0]])
    
    # Compute e_prime_matrix (skew-symmetric matrix of e_prime)
    e_prime_matrix = np.array([
        [0, -e_prime[2], e_prime[1]],
        [e_prime[2], 0, -e_prime[0]],
        [-e_prime[1], e_prime[0], 0]
    ])
    
    # Compute P' as [e_prime_matrix * F | e_prime]
    P_prime = np.hstack((e_prime_matrix @ F, e_prime.reshape(3, 1)))

    # Ensure that both matrices have a rank of 3
    assert np.linalg.matrix_rank(P) == 3, "P does not have full rank"
    assert np.linalg.matrix_rank(P_prime) == 3, "P_prime does not have full rank"
    
    return P, P_prime


e, e_prime = compute_epipoles(F)
P, P_prime = compute_projection_matrices(e_prime, F)

print("Left Epipole (e):", e)
print("Right Epipole (e'):", e_prime)
print("Projection Matrix P:\n", P)
print("Projection Matrix P':\n", P_prime)


In [ ]:
def triangulate(P, P_prime, x_points, x_prime_points):
    world_points = []
    for x, x_prime in zip(x_points, x_prime_points):
        A = np.zeros((4, 4))
        A[0] = x[0] * P[2] - P[0]
        A[1] = x[1] * P[2] - P[1]
        A[2] = x_prime[0] * P_prime[2] - P_prime[0]
        A[3] = x_prime[1] * P_prime[2] - P_prime[1]
        
        _, _, vh = np.linalg.svd(A)
        world_point = vh[-1]
        world_points.append(world_point / world_point[3])  # Convert to homogeneous coordinates

    return np.array(world_points)[:, :3]

def world2image(P, x):
    x_h = np.append(x, 1)
    img_point = P @ x_h
    return img_point[:2] / img_point[2]

def cost_function(params, x_points, x_prime_points):
    # Reshape params into P_prime
    P_prime = params.reshape(3, 4)
    
    # Project points and calculate residuals
    world_points = triangulate(P, P_prime, x_points, x_prime_points)
    x_prime_projected = np.array([world2image(P_prime, pt) for pt in world_points])
    residuals = (x_prime_points - x_prime_projected).ravel()
    return residuals

# Perform LM optimization to refine P_prime
initial_params = P_prime.ravel()
result = least_squares(cost_function, initial_params, args=(pic_1_points, pic_2_points), method='lm')




In [ ]:
P_prime_optimized = result.x.reshape(3, 4)
print("Optimized Projection Matrix for Right Image (P_prime):\n", P_prime_optimized)


def refine_translation(P_prime_ref):
    # Step 1: Compute t_ref from the refined projection matrix
    t_ref = P_prime_ref[:, -1]

    # Step 2: Generate the skew-symmetric matrix for t_ref
    t_ref_matrix = np.array([
        [0, -t_ref[2], t_ref[1]],
        [t_ref[2], 0, -t_ref[0]],
        [-t_ref[1], t_ref[0], 0]
    ])

    # Step 3: Compute the refined fundamental matrix F_ref
    F_ref = t_ref_matrix @ P_prime_ref[:, :3]
    F_ref = F_ref / F_ref[-1, -1]  # Normalize

    # Step 4: Compute the refined right epipole e_prime_ref
    e_prime_ref = compute_epipoles(F_ref)[1]
    return e_prime_ref

e_prime_optimized =refine_translation(P_prime_optimized)

In [ ]:
def get_H_prime(img, e_prime):


    h, w = img.shape[:2]

    # Compute T1
    T1 = np.array([[1.0, 0.0, -w/2],
                   [0.0, 1.0, -h/2],
                   [0.0, 0.0, 1.0]])

    # Compute T2
    T2 = np.array([[1.0, 0.0, w/2],
                   [0.0, 1.0, h/2],
                   [0.0, 0.0, 1.0]])

    # Compute x0, y0, and theta
    x0 = w/2
    y0 = h/2
    e_prime = e_prime / e_prime[-1]
    ex = e_prime[0]
    ey = e_prime[1]
    theta = np.arctan(-(ey-y0)/(ex-x0))

    # Compute R and f
    R = np.array([[np.cos(theta), -np.sin(theta), 0],
                   [np.sin(theta), np.cos(theta), 0],
                   [0, 0, 1]])
    f = np.abs((ex-x0)*np.cos(theta) - (ey-y0)*np.sin(theta))

    # Compute G
    G = np.array([[1, 0, 0],
                   [0, 1, 0],
                   [-1/f, 0, 1]])

    # Compute H_prime
    H_prime = np.matmul(T2, np.matmul(G, np.matmul(R, T1)))

    return H_prime, f

H_right, f_right=get_H_prime(img2, e_prime_optimized)
print(H_right)

In [ ]:

def rectify_left_image(P, P_prime, H_prime, x_points, x_prime_points):

    # Compute the pseudo-inverse of P
    P_pinv = np.linalg.pinv(P)

    # Step 1: Compute the initial homography H_0
    H_0 = np.matmul(P_prime, P_pinv)

    # Step 2: Transform points in the left image using H_0
    x_hat = np.array([np.dot(H_0, np.hstack((x, 1))) for x in x_points])
    x_hat = (x_hat.T / x_hat[:, 2]).T  # Normalize to homogeneous coordinates

    # Step 3: Transform points in the right image using H_prime
    x_prime_hat = np.array([np.dot(H_prime, np.hstack((x_prime, 1))) for x_prime in x_prime_points])
    x_prime_hat = (x_prime_hat.T / x_prime_hat[:, 2]).T  # Normalize to homogeneous coordinates

    # Step 4: Solve for affine transformation H_a to align x_hat with x_prime_hat
    A = []
    b = []
    for i in range(len(x_hat)):
        x, y, _ = x_hat[i]
        x_prime, y_prime, _ = x_prime_hat[i]

        # Create equations for x and y coordinates
        A.append([x, y, 1, 0, 0, 0])
        A.append([0, 0, 0, x, y, 1])
        b.append(x_prime)
        b.append(y_prime)

    A = np.array(A)
    b = np.array(b)

    # Solve for the affine transformation parameters
    a, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
    
    # Step 5: Construct H_a as an affine transformation matrix
    H_a = np.array([
        [a[0], a[1], a[2]],
        [a[3], a[4], a[5]],
        [0, 0, 1]
    ])

    # Step 6: Compute the final homography H for the left image
    H = H_a @ H_0
    return H

H_left = rectify_left_image(P, P_prime_optimized,H_right, pic_1_points, pic_2_points)
print(H_left)

In [ ]:
def rectify_images(img1, img2, H, Hp, w=3024, h=4032):


    # Rectify images using cv2.warpPerspective
    rectified1 = cv2.warpPerspective(img1, H, (w, h))
    rectified2 = cv2.warpPerspective(img2, Hp, (w, h))
    # print(rectified1)
    # Display the rectified images
    plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(rectified1, cv2.COLOR_BGR2RGB))
    plt.title('Rectified Image 1')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(rectified2, cv2.COLOR_BGR2RGB))
    plt.title('Rectified Image 2')
    plt.axis('off')
    plt.savefig('Rectified correspondence')
    plt.show()

    return rectified1, rectified2

rectified_img1, rectified_img2 = rectify_images(img1, img2, H_left, H_right)



In [ ]:

def rectify_images_with_correspondences(img1, img2, H, Hp, points1, points2, w, h):


    # Rectify images using cv2.warpPerspective
    rectified1 = cv2.warpPerspective(img1, H, (w, h))
    rectified2 = cv2.warpPerspective(img2, Hp, (w, h))

    # Apply homography to points to get rectified points
    points1_rectified = cv2.perspectiveTransform(np.array([points1], dtype='float32'), H)[0]
    points2_rectified = cv2.perspectiveTransform(np.array([points2], dtype='float32'), Hp)[0]

    # Plot rectified images side by side with correspondences
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(cv2.cvtColor(rectified1, cv2.COLOR_BGR2RGB))
    ax[0].set_title('Rectified Image 1')
    ax[0].axis('off')

    ax[1].imshow(cv2.cvtColor(rectified2, cv2.COLOR_BGR2RGB))
    ax[1].set_title('Rectified Image 2')
    ax[1].axis('off')

    # Draw lines for corresponding points
    for pt1, pt2 in zip(points1_rectified, points2_rectified):
        # Convert points to integers
        pt1 = tuple(map(int, pt1))
        pt2 = tuple(map(int, pt2))

        # Draw a circle for the point in both images
        ax[0].plot(pt1[0], pt1[1], 'ro')  # red point in left image
        ax[1].plot(pt2[0], pt2[1], 'ro')  # red point in right image

        # Draw a line on both images to show correspondence (across same row if rectified correctly)
        ax[0].hlines(pt1[1], 0, w, colors='blue', linestyles='dotted', linewidth=1.5)
        ax[1].hlines(pt2[1], 0, w, colors='blue', linestyles='dotted', linewidth=1.5)

    plt.tight_layout()
    plt.savefig('Rectified correspondence epipolar')
    plt.show()

    return points1_rectified, points2_rectified, rectified1, rectified2

rectified_point1, rectified_point2, rectified_img1, rectified_img2 = rectify_images_with_correspondences(img1, img2, H_left, H_right, pic_1_points, pic_2_points, 3024, 4032)


In [ ]:

def find_correspondences(rectified_img1, rectified_img2, points1_rectified, search_window=5):

    h, w, _ = rectified_img2.shape
    correspondences = []

    for pt1 in points1_rectified:
        x1, y1 = int(pt1[0]), int(pt1[1])

        # Define a search window in the second image
        y2_start = max(0, y1 - search_window)
        y2_end = min(h, y1 + search_window + 1)

        # Extract the search row and the corresponding row from both images
        row1 = rectified_img1[y1, :]
        row2 = rectified_img2[y2_start:y2_end, :]

        # Calculate SSD for each pixel in the search window
        best_ssd = float('inf')
        best_x2 = x1  # Default to the same x-coordinate if no match is found
        for y2_idx, y2 in enumerate(range(y2_start, y2_end)):
            ssd = np.sum((row1[x1 - 5:x1 + 5] - row2[y2_idx, x1 - 5:x1 + 5]) ** 2)
            if ssd < best_ssd:
                best_ssd = ssd
                best_x2 = x1  # Corresponding x-coordinate (for rectified images)

        correspondences.append((best_x2, y1))

    return correspondences



def draw_correspondences(rectified_img1, rectified_img2, points1_rectified, corresponding_points):

    # Combine both images side-by-side
    combined_img = np.hstack((rectified_img1, rectified_img2))

    # Offset for the second image (to adjust x-coordinates when drawing lines)
    offset_x = rectified_img1.shape[1]

    # Convert to RGB if the images are grayscale
    if len(combined_img.shape) == 2:
        combined_img = cv2.cvtColor(combined_img, cv2.COLOR_GRAY2BGR)

    # Draw points and lines
    for pt1, pt2 in zip(points1_rectified, corresponding_points):
        x1, y1 = int(pt1[0]), int(pt1[1])
        x2, y2 = int(pt2[0] + offset_x), int(pt2[1])  # Offset x-coordinate for the second image

        # Draw points
        cv2.circle(combined_img, (x1, y1), 5, (0, 255, 0), -1)  # Green for points in Image 1
        cv2.circle(combined_img, (x2, y2), 5, (255, 0, 0), -1)  # Blue for points in Image 2
        
        bgr_color = (np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256))

        # Draw connecting lines
        cv2.line(combined_img, (x1, y1), (x2, y2), bgr_color, 3)  # Yellow lines

    # Display the combined image with correspondences
    plt.figure(figsize=(12, 6))
    plt.imshow(cv2.cvtColor(combined_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Correspondences Between Rectified Images")
    plt.savefig(("Rectified Images canny"))
    plt.show()


interest_points = cv2.goodFeaturesToTrack(cv2.cvtColor(rectified_img1, cv2.COLOR_BGR2GRAY), 500, 0.01, 10)
interest_points = np.int0(interest_points).reshape(-1, 2)

# Find correspondences in the second rectified image
corresponding_points = find_correspondences(rectified_img1, rectified_img2, interest_points)

# Visualize correspondences with lines
draw_correspondences(rectified_img1, rectified_img2, interest_points, corresponding_points)


In [ ]:



def back_project_to_world(canny_points1, canny_points2, P_left, P_right):

    def triangulate_point(pt1, pt2, P1, P2):
        """Triangulate a single pair of points."""
        A = np.zeros((4, 4))
        A[0] = pt1[0] * P1[2] - P1[0]
        A[1] = pt1[1] * P1[2] - P1[1]
        A[2] = pt2[0] * P2[2] - P2[0]
        A[3] = pt2[1] * P2[2] - P2[1]

        _, _, vh = np.linalg.svd(A)
        world_point = vh[-1]
        return world_point[:3] / world_point[3]  # Convert to 3D (inhomogeneous) coordinates

    world_points = []
    for pt1, pt2 in zip(canny_points1, canny_points2):
        world_point = triangulate_point(pt1, pt2, P_left, P_right)
        world_points.append(world_point)

    return np.array(world_points)

world_points = back_project_to_world(rectified_point1, rectified_point2, P, P_prime_optimized)


e_n, e_prime_n = compute_epipoles(F_n)
P_n, P_prime_n = compute_projection_matrices(e_prime_n, F_n)

# Perform LM optimization to refine P_prime
initial_params = P_prime_n.ravel()
result_n = least_squares(cost_function, initial_params, args=(interest_points, corresponding_points), method='lm')
P_prime_optimized_n = result_n.x.reshape(3, 4)
print("Optimized Projection Matrix for Right Image (P_prime):\n", P_prime_optimized_n)

# Back-project points into 3D world space
world_points = back_project_to_world(pic_1_points, pic_2_points, P_n, P_prime_optimized_n)

# Display the resulting 3D points
# print("3D World Points:")
# print(world_points)

    


In [ ]:

image1_points = pic_1_points
image2_points = pic_2_points


# Convert images from BGR (OpenCV default) to RGB for Matplotlib
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

# Visualization
fig = plt.figure(figsize=(12, 10))

# Plot Image 1 with points
ax1 = fig.add_subplot(3, 2, 1)
ax1.imshow(img1)
ax1.scatter(image1_points[:, 0], image1_points[:, 1], color='r', label='Image 1 Points')
ax1.set_title("Image 1")
ax1.axis('off')

# Plot Image 2 with points
ax2 = fig.add_subplot(3, 2, 2)
ax2.imshow(img2)
ax2.scatter(image2_points[:, 0], image2_points[:, 1], color='g', label='Image 2 Points')
ax2.set_title("Image 2")
ax2.axis('off')

# Combine both images with corresponding lines
ax3 = fig.add_subplot(3, 1, 2)
ax3.imshow(np.hstack((img1, img2)))
offset = img1.shape[1]  # Offset for the second image
for i in range(len(image1_points)):
    ax3.plot(
        [image1_points[i, 0], image2_points[i, 0] + offset],
        [image1_points[i, 1], image2_points[i, 1]],
        color='b'
    )
ax3.scatter(image1_points[:, 0], image1_points[:, 1], color='r', label='Image 1 Points')
ax3.scatter(image2_points[:, 0] + offset, image2_points[:, 1], color='g', label='Image 2 Points')
ax3.set_title("Image Correspondences")
ax3.axis('off')

# 3D Plot of reconstructed points
ax4 = fig.add_subplot(3, 1, 3, projection='3d')
ax4.scatter(world_points[:, 0], world_points[:, 1], world_points[:, 2], color='m', label='World Points')

# Draw lines from world points to the corresponding image points
for i in range(len(world_points)):
    # Line to Image 1
    ax4.plot(
        [world_points[i, 0], -4],  # Arbitrary connections to image plane
        [world_points[i, 1], -3],
        [world_points[i, 2], -1.5],
        color='r'
    )
    # Line to Image 2
    ax4.plot(
        [world_points[i, 0], -4],
        [world_points[i, 1], -3],
        [world_points[i, 2], -1.5],
        color='g'
    )

ax4.set_title("3D Reconstruction")
ax4.set_xlabel("X")
ax4.set_ylabel("Y")
ax4.set_zlabel("Z")
ax4.legend()

plt.tight_layout()
plt.show()

In [ ]:

def apply_census(imgL, imgR, dMax):

    """ Apply the census transform to estimate the disparity map. """

    # Ensure we use grayscale images
    if len(imgL.shape) > 2:
        imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    if len(imgR.shape) > 2:
        imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)

    halfWinSz = int(M / 2)
    borderSz = dMax + halfWinSz

    height, width = imgL.shape
    dMap = np.zeros((height, width), dtype=np.uint8)

    # Optimize by reducing window size access repetition and loop restructuring
    for rowL in range(borderSz, height - borderSz):
        for colL in range(borderSz, width - borderSz):
            windowL = imgL[rowL - halfWinSz:rowL + halfWinSz + 1, colL - halfWinSz:colL + halfWinSz + 1].ravel()
            minCost = float('inf')
            bestMatch = 0

            # Iterate over disparity values to find best match
            for d in range(dMax + 1):
                colR = colL - d
                if colR < borderSz:
                    break

                windowR = imgR[rowL - halfWinSz:rowL + halfWinSz + 1, colR - halfWinSz:colR + halfWinSz + 1].ravel()
                cost = np.sum(windowL != windowR)  # Hamming distance as a simple matching cost

                if cost < minCost:
                    minCost = cost
                    bestMatch = d

            dMap[rowL, colL] = bestMatch

    return dMap
    # Implement the census transform here (or use existing function)
    # This is a placeholder for census-based stereo matching
    # Here, we assume that this function returns a disparity map
    

# Load images and ground truth disparity map
imgL = cv2.imread('Task3Images/Task3Images/im2.png', cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread('Task3Images/Task3Images/im6.png', cv2.IMREAD_GRAYSCALE)
gtDMap = cv2.imread('Task3Images/Task3Images/disp2.png', cv2.IMREAD_GRAYSCALE)

# Convert ground truth disparity map to float32 and adjust
gtDMap = gtDMap.astype(np.float32)
gtDMap /= 4  # Adjust by dividing by 4 as per the instruction
gtDMap = gtDMap.astype(np.uint8)
dMax = int(gtDMap.max())
M=31
# Estimate disparity map using Census transform
dMap = apply_census(imgL, imgR, dMax)

# Normalize dMap for viewing
dMapView = (dMap / np.max(dMap) * 255).astype(np.uint8)

# Compute the error map
error = np.abs(dMap.astype(np.int16) - gtDMap.astype(np.int16)).astype(np.uint8)

# Binary mask where error is <= delta
delta = 2  # You can adjust the delta value as needed
errorMask = np.zeros_like(error)
errorMask[error <= delta] = 255  # Set mask value to 255 where error is <= delta

# Compute the number of valid pixels (non-black pixels in ground truth)
validMask = gtDMap > 0  # Pixels where ground truth disparity is valid
N = np.count_nonzero(validMask)  # Total number of valid pixels

# Compute the d1 and d2 error ratios
validError = error[validMask]
d1Error = np.count_nonzero(validError > 1) / N
d2Error = np.count_nonzero(validError > 2) / N

# Compute the accuracy
correctMatches = np.count_nonzero((error <= delta) & validMask)  # Pixels with error <= delta
accuracy = correctMatches / N  # Ratio of correct matches to valid pixels

print(f'd1 Error: {d1Error:.2%}')
print(f'd2 Error: {d2Error:.2%}')
print(f'Accuracy: {accuracy:.2%}')

# Visualize the results
plt.figure(figsize=(10, 8))

# Show the disparity map
plt.subplot(121)
plt.imshow(dMapView, cmap='gray')
plt.title('Estimated Disparity Map')
plt.axis('off')

# Show the error mask (highlighting challenging regions)
plt.subplot(122)
plt.imshow(errorMask, cmap='gray')
plt.title('Binary Error Mask')
plt.axis('off')

plt.tight_layout()
plt.show()

cv2.imwrite(f'error_mask M={M}, delta= {delta} d1d2= {d1Error, d2Error}.png', errorMask)
cv2.imwrite(f'Disparity map M={M}, delta= {delta} d1d2= {d1Error, d2Error}.png', dMapView)

